In [1]:
import pandas as pd

# Loading the dataset
data = pd.read_csv('data/product_info.csv')

In [12]:
data.columns

Index(['product_id', 'product_name', 'brand_id', 'brand_name', 'loves_count',
       'rating', 'reviews', 'size', 'variation_type', 'variation_value',
       'variation_desc', 'ingredients', 'price_usd', 'value_price_usd',
       'sale_price_usd', 'limited_edition', 'new', 'online_only',
       'out_of_stock', 'sephora_exclusive', 'highlights', 'primary_category',
       'secondary_category', 'tertiary_category', 'child_count',
       'child_max_price', 'child_min_price'],
      dtype='object')

In [5]:
# The only required columns
data = data[['product_id', 'brand_name', 'product_name', 'ingredients', 'primary_category',
       'secondary_category', 'tertiary_category']]

# Checking for missing values
data.isnull().sum()


product_id              0
brand_name              0
product_name            0
ingredients             0
primary_category        0
secondary_category      4
tertiary_category     907
dtype: int64

In [2]:
# Dropping null values
data.dropna(subset=['ingredients'], inplace=True)

In [3]:
# Counting duplicates
duplicate_count = data.duplicated(keep=False).sum()
print(f"Number of duplicate rows: {duplicate_count}")

Number of duplicate rows: 0


In [4]:
data['primary_category'].value_counts()

primary_category
Skincare           2286
Makeup             2033
Hair               1274
Fragrance          1255
Bath & Body         374
Mini Size           266
Men                  59
Tools & Brushes       2
Name: count, dtype: int64

In [6]:
# Choosing 4 main primary categories and excluding the others
data = data[data['primary_category'].isin(['Skincare', 'Makeup', 'Hair', 'Fragrance'])]

Cleaning the Ingredients column

In [7]:
# Clean the ingredients column
data['ingredients'] = data['ingredients'].str.strip()  # Remove leading and trailing spaces
data['ingredients'] = data['ingredients'].str.replace(r'^[^\w]+|[^\w]+$', '', regex=True)  # Remove unwanted symbols


In [9]:
from collections import Counter

# Step 1: Split the ingredients into individual items
ingredient_lists = data['ingredients'].str.split(',')

# Step 2: Flatten the list of all ingredients
all_ingredients = [ingredient.strip() for sublist in ingredient_lists for ingredient in sublist]

# Step 3: Count occurrences of each ingredient
ingredient_counts = Counter(all_ingredients)

# Step 4: Display the most common ingredients
most_common_ingredients = ingredient_counts.most_common()

# Print the results
print("Most Common Ingredients:")
for ingredient, count in most_common_ingredients:
    print(f"{ingredient}: {count}")

Most Common Ingredients:
Glycerin: 3814
Phenoxyethanol: 3789
Tocopherol: 2649
Caprylyl Glycol: 2579
Linalool: 2523
Limonene: 2506
Ethylhexylglycerin: 2280
Silica: 2175
Citric Acid: 2155
Dimethicone: 2131
Butylene Glycol: 2128
Potassium Sorbate: 1787
Sodium Benzoate: 1657
Citronellol: 1634
Caprylic/Capric Triglyceride: 1588
Geraniol: 1571
Tocopheryl Acetate: 1523
Propanediol: 1497
Water: 1494
Sodium Hyaluronate: 1451
Mica: 1417
Citral: 1368
Benzyl Alcohol: 1315
Xanthan Gum: 1203
Sodium Hydroxide: 1187
Cetearyl Alcohol: 1024
Panthenol: 1019
Benzyl Salicylate: 998
1: 973
Pentylene Glycol: 964
Squalane: 946
Tin Oxide: 935
Benzyl Benzoate: 886
Fragrance: 870
2-Hexanediol: 859
Hexyl Cinnamal: 844
Zinc Stearate: 843
Aqua/Water/Eau: 838
Coumarin: 836
Hydroxycitronellal: 827
Alcohol: 824
Alcohol Denat.: 798
Sodium Chloride: 786
Synthetic Fluorphlogopite: 752
Ci 77492: 750
Water/Aqua/Eau: 748
Glyceryl Stearate: 744
Helianthus Annuus (Sunflower) Seed Oil: 744
Parfum (Fragrance): 717
Parfum/Fragra

Creating an allergens column in the dataset

In [9]:
# Define the path to your text file in Google Drive
harmful_ingredients_path = 'data/harmful_ingredients.txt'  # Adjust the path

# Load the text file and split by comma to get the list of harmful ingredients
with open(harmful_ingredients_path, 'r') as file:
    harmful_ingredients = file.read().split(',')

# Strip any extra whitespace from each ingredient
harmful_ingredients = [ingredient.strip() for ingredient in harmful_ingredients]

# Ensure the ingredients column is in a string format
data['ingredients'] = data['ingredients'].astype(str)

# Tokenize each ingredient list in the dataset by splitting on commas
data['ingredients_list'] = data['ingredients'].apply(lambda x: [ingredient.strip() for ingredient in x.split(',')])

# View the preprocessed data
data[['product_name', 'ingredients_list']].head()

# Create a new column to store detected harmful ingredients
data['harmful_detected'] = data['ingredients_list'].apply(
    lambda ingredients: [ingredient for ingredient in ingredients if ingredient in harmful_ingredients]
)

# View the results
data[['product_name', 'ingredients_list', 'harmful_detected']].head()

# Count harmful ingredients for each product
data['harmful_count'] = data['harmful_detected'].apply(len)

# View the results
data[['product_name', 'harmful_detected', 'harmful_count']].head()

,product_name,harmful_detected,harmful_count
0,Fragrance Discovery Set,"[Linalool, Citral, Geraniol, Eugenol, Farnesol...",27
1,La Habana Eau de Parfum,"[Citral, Eugenol, Linalool]",3
2,Rainbow Bar Eau de Parfum,"[Linalool, Coumarin, Eugenol, Citral, Isoeugen...",6
3,Kasbah Eau de Parfum,"[Eugenol, Linalool, Citronellol, Geraniol, Cit...",5
4,Purple Haze Eau de Parfum,"[Geraniol, Linalool, Coumarin, Farnesol, Eugen...",6


In [24]:
# Exporting a new dataset
data.to_csv('data/new_product_info.csv', index=False)